 # Data_Cleaning

In [2]:
import pandas as pd
import numpy as np

data=pd.read_csv(r"C:\Users\sohaibkhan\Documents\Pakistan Largest Ecommerce Dataset.csv[1]\Pakistan Largest Ecommerce Dataset.csv")
data.head(3)

# -----------------------------
# 1. Drop completely irrelevant columns
# -----------------------------
drop_cols = [
    "Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25",
    "item_id", "sku", "increment_id", "Customer ID",
    "sales_commission_code", "BI Status", "Customer Since", "M-Y", "FY", " MV "
]
data.drop(columns=drop_cols, inplace=True, errors='ignore')

# -----------------------------
# 2. Convert date columns to datetime
# -----------------------------
data['created_at'] = pd.to_datetime(data['created_at'], errors='coerce')
data['Working Date'] = pd.to_datetime(data['Working Date'], errors='coerce')

# -----------------------------
# 3. Extract useful date features BEFORE dropping the original dates
# -----------------------------
data['year'] = data['created_at'].dt.year
data['month'] = data['created_at'].dt.month
data['dayofweek'] = data['created_at'].dt.dayofweek
data['is_weekend'] = data['dayofweek'].isin([5, 6]).astype(int)
data['quarter'] = data['created_at'].dt.quarter

# -----------------------------
# 4. Drop rows where target 'price' is missing
# -----------------------------
data = data[~data['price'].isnull()]

# -----------------------------
# 5. Handle missing values for numeric columns
# -----------------------------
num_cols = ['qty_ordered', 'discount_amount', 'grand_total', 'year', 'month']
for col in num_cols:
    data[col] = data[col].fillna(0)

# -----------------------------
# 6. Handle missing values for categorical columns
# -----------------------------
cat_cols = ['category_name_1', 'payment_method']
for col in cat_cols:
    data[col] = data[col].fillna('Unknown')

# -----------------------------
# 7. Feature Engineering
# -----------------------------

# Remove target leakage: drop grand_total because it’s derived from price
if 'grand_total' in data.columns:
    data.drop(columns=['grand_total'], inplace=True)

# Discount per unit (avoid division by zero)
data['discount_per_unit'] = np.where(
    data['qty_ordered'] > 0,
    data['discount_amount'] / data['qty_ordered'],
    0
)

# Is discounted (binary feature)
data['is_discounted'] = (data['discount_amount'] > 0).astype(int)

# Log-transform skewed features
for col in ['price', 'qty_ordered', 'discount_amount', 'discount_per_unit']:
    data[col] = np.log1p(data[col])

# -----------------------------
# 8. Final check
# -----------------------------
print(data.info())
print(data.isnull().sum())
print(data.head())

# Save cleaned dataset
data.to_csv("cleaned_ready_for_modeling.csv", index=False)


C:\Users\sohaibkhan\AppData\Local\Temp\ipykernel_2248\904549278.py:4: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r"C:\Users\sohaibkhan\Documents\Pakistan Largest Ecommerce Dataset.csv[1]\Pakistan Largest Ecommerce Dataset.csv")
D:\6 Semester\anacondala\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\6 Semester\anacondala\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


<class 'pandas.core.frame.DataFrame'>
Index: 584524 entries, 0 to 584523
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   status             584509 non-null  object        
 1   created_at         584524 non-null  datetime64[ns]
 2   price              584524 non-null  float64       
 3   qty_ordered        584524 non-null  float64       
 4   category_name_1    584524 non-null  object        
 5   discount_amount    584521 non-null  float64       
 6   payment_method     584524 non-null  object        
 7   Working Date       584524 non-null  datetime64[ns]
 8   Year               584524 non-null  float64       
 9   Month              584524 non-null  float64       
 10  year               584524 non-null  float64       
 11  month              584524 non-null  float64       
 12  dayofweek          584524 non-null  float64       
 13  is_weekend         584524 non-null  int32        

In [3]:
import joblib

In [4]:
joblib.dump(data,"C:/Users/sohaibkhan/Project1/Data/DataCleaning.pkl")

['C:/Users/sohaibkhan/Project1/Data/DataCleaning.pkl']

In [5]:
print("Training features:", list(data.columns))

Training features: ['status', 'created_at', 'price', 'qty_ordered', 'category_name_1', 'discount_amount', 'payment_method', 'Working Date', 'Year', 'Month', 'year', 'month', 'dayofweek', 'is_weekend', 'quarter', 'discount_per_unit', 'is_discounted']
